que (a). Build a classifier to predict the outcome of a new patient with high accuracy. Also, remember
that as a data-scientist working on healthcare problems, your intent should also be to minimize
the number of false-negatives.

IMPORT DEPENDENCIES

In [293]:
import pandas as pd
import numpy as np
import math
import os
from pandas import read_excel

LOAD DATASET

In [349]:
file_name  ="/home/kgs/image caption/BreastCancer_Prognostic_v1.xlsx" 
data = read_excel(file_name)
data.head(5)

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


ASSIGNING BINARY VALUE TO OUTCOME (R=1, N=0)

In [351]:
diagnosis = pd.get_dummies(data['Outcome'], drop_first=True)
data = pd.concat([data,diagnosis],axis=1)
data.head(5)

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status,N,R,R
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5,1,0,0
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2,1,0,0
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0,1,0,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0,1,0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0,0,1,1


CHECKING FOR NAN VALUES


In [296]:
data.isnull()

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status,R
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
194,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
195,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
196,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [297]:
data.isnull().sum()


ID                           0
Outcome                      0
Time                         0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave_points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_std_dev               0
texture_std_dev              0
perimeter_std_dev            0
area_std_dev                 0
smoothness_std_dev           0
compactness_std_dev          0
concavity_std_dev            0
concave_points_std_dev       0
symmetry_std_dev             0
fractal_dimension_std_dev    0
Worst_radius                 0
Worst_texture                0
Worst_perimeter              0
Worst_area                   0
Worst_smoothness             0
Worst_compactness            0
Worst_concavity              0
Worst_concave_points         0
Worst_symmetry               0
Worst_fr

In [298]:
len(data)

198

In [299]:
k = '?'#found it in dataset so have to remove it
data1 = data[data['Lymph_Node_Status'] != k ]

In [300]:
data1.head()
data2 = data1

In [301]:
len(data1)

194

In [302]:
Lymph_Node_Status = data1["Lymph_Node_Status"]
for i in Lymph_Node_Status:
    print(i)# haven't any ? values Lymph_Node_Status:
    

5
2
0
0
0
0
10
1
20
0
0
0
6
0
1
0
1
0
1
0
13
0
0
2
1
0
0
13
10
0
0
0
0
1
1
0
1
0
13
6
0
1
0
4
2
0
1
0
2
1
0
0
4
2
1
17
0
15
11
0
9
0
8
1
0
7
2
0
3
1
2
1
1
4
7
1
0
3
0
4
9
0
1
14
0
0
0
1
0
2
0
0
1
27
5
24
1
0
1
0
7
0
15
0
0
3
1
1
0
6
2
11
0
0
15
0
18
0
11
0
1
2
2
0
0
4
13
0
0
0
0
2
1
0
0
0
13
16
3
13
0
1
27
0
4
0
0
7
0
7
0
0
9
0
2
0
20
0
4
1
8
1
4
1
1
0
0
0
0
1
1
2
0
9
4
0
2
0
0
4
0
0
0
0
1
7
2
0
1
21
2
0
0
0


DROP UNNECESSARY COLUMN 

In [303]:
#AS WE HAVE CHANGE OUTCOME TO BINARY AS COLUMN 'R' AND THERE WILL BE NO NEED OF ID
data1.drop(['Outcome', 'ID'],axis=1,inplace=True)

/home/kgs/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [304]:
data1.head(5)

,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status,R
0,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5,0
1,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2,0
2,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0,0
3,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0,0
4,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0,1


DECIDING TARGET VARIBLE AND BREAK IT INTO X AND Y

In [315]:
    #HERE WE HAVE "R" AS TARGET TO WEATHER IT IS RECURRENT OR NON-RECURRENT. SO,
X=data1.drop("R", axis=1)
y=data1["R"]
#datak = data1

SPLITTING DATASET 70/30

In [353]:
from sklearn.model_selection import train_test_split

In [354]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [355]:
datak = X_test

USING SVM HERE

In [356]:
import warnings
from sklearn.svm import SVC

In [357]:
model = SVC(C=1.3, kernel='rbf')

FIT DATA TO MODEL

In [358]:
model.fit(X_train, y_train)

/home/kgs/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.3, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [359]:
predictions = model.predict(X_test)

In [360]:
len(predictions) 

59

In [361]:
datak['prediction'] = predictions

/home/kgs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [362]:
datak['y_test'] = y_test

/home/kgs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [363]:
datak

,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status,prediction,y_test
46,26,13.80,15.79,90.43,584.1,0.10070,0.12800,0.07789,0.05069,0.1662,...,0.14110,0.35420,0.27790,0.13830,0.2589,0.10300,3.0,2,0,1
175,29,21.93,30.64,146.70,1487.0,0.08679,0.17230,0.20530,0.10100,0.1796,...,0.11680,0.40720,0.44940,0.18860,0.2784,0.07353,3.5,0,0,0
166,39,24.63,21.60,165.50,1841.0,0.10300,0.21060,0.23100,0.14710,0.1991,...,0.13420,0.41880,0.46580,0.24750,0.3157,0.09671,1.0,1,0,0
37,8,18.65,17.60,123.70,1076.0,0.10990,0.16860,0.19740,0.10090,0.1907,...,0.16790,0.50900,0.73450,0.23780,0.3799,0.09185,1.8,0,0,1
139,13,19.55,23.21,128.90,1174.0,0.10100,0.13180,0.18560,0.10210,0.1989,...,0.12510,0.24140,0.38290,0.18250,0.2576,0.07602,4.0,13,0,0
12,117,15.85,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,...,0.11310,0.19240,0.23220,0.11190,0.2809,0.06287,1.0,0,0,0
126,34,17.06,21.00,111.80,918.6,0.11190,0.10560,0.15080,0.09934,0.1727,...,0.14490,0.20530,0.39200,0.18270,0.2623,0.07599,1.5,0,0,0
84,5,19.18,22.49,127.50,1148.0,0.08523,0.14280,0.11140,0.06772,0.1767,...,0.13220,0.56010,0.38650,0.17080,0.3193,0.09221,3.0,1,0,0
178,24,24.29,25.48,161.80,1715.0,0.09374,0.22840,0.27020,0.13690,0.2307,...,0.12470,0.39350,0.61180,0.20630,0.3983,0.07978,1.2,0,0,0
105,19,19.55,28.77,133.60,1207.0,0.09260,0.20630,0.17840,0.11440,0.1893,...,0.12810,0.53290,0.42510,0.19410,0.2818,0.10050,6.0,15,0,1


CONCATINATION AND WRITING IT TO PREDICTIONS.XLSX

In [364]:
writer = pd.ExcelWriter('predictions.xlsx', engine='xlsxwriter')
datak.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [365]:
from sklearn.metrics import classification_report

In [366]:
classification_report(y_test,predictions )

/home/kgs/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'              precision    recall  f1-score   support\n\n           0       0.71      1.00      0.83        42\n           1       0.00      0.00      0.00        17\n\n    accuracy                           0.71        59\n   macro avg       0.36      0.50      0.42        59\nweighted avg       0.51      0.71      0.59        59\n'

In [367]:
from sklearn.metrics import confusion_matrix

In [368]:
confusion_matrix(y_test, predictions)

array([[42,  0],
       [17,  0]])

In [369]:
from sklearn.metrics import accuracy_score

In [370]:
accuracy_score(y_test,predictions)

0.711864406779661

I HAVE ALSO TRIED LOGISTIC REGRESSION BUT SVM IS PERFORMING GOOD IN THIS CASE FOR CLASSIFICATION 

que(b). Build a regression model to predict the recurrence time for patients whose outcome is R.

In [242]:
file_name  ="/home/kgs/image caption/BreastCancer_Prognostic_v1.xlsx" 
dataset = read_excel(file_name)
dataset.head(5)

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [249]:
len(dataset)

198

In [250]:
k = '?'#found it in dataset so have to remove it
dataset1 = dataset[dataset['Lymph_Node_Status'] != k ]

In [251]:
k="R"
len(dataset1)

194

In [252]:
dataset1.head(5)


,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [253]:
df = dataset1[dataset1['Outcome'] == k ]

In [254]:
df.head(5)

,ID,Outcome,Time,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,Worst_perimeter,Worst_area,Worst_smoothness,Worst_compactness,Worst_concavity,Worst_concave_points,Worst_symmetry,Worst_fractal_dimension,Tumor_Size,Lymph_Node_Status
4,843584,R,27,20.29,14.34,135.10,1297.0,0.1003,0.1328,0.19800,...,152.2,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0
5,843786,R,77,12.75,15.29,84.60,502.7,0.1189,0.1569,0.16640,...,107.3,733.2,0.1706,0.4196,0.5999,0.1709,0.3485,0.11790,2.5,0
7,844582,R,77,13.71,20.83,90.20,577.9,0.1189,0.1645,0.09366,...,110.6,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,4.0,10
13,847990,R,36,14.54,27.54,96.73,658.8,0.1139,0.1595,0.16390,...,124.1,943.2,0.1678,0.6577,0.7026,0.1712,0.4218,0.13410,6.0,6
15,848620,R,10,16.13,20.68,108.10,798.8,0.1170,0.2022,0.17220,...,136.8,1315.0,0.1789,0.4233,0.4784,0.2073,0.3706,0.11420,3.0,1


IMPORTING DEPENDENCIES FOR LINEAR REGRESSION

In [255]:
from sklearn.linear_model import LinearRegression

In [256]:
y = df["Time"]

In [258]:
df.drop(['Outcome', 'ID' ],axis=1,inplace=True)

/home/kgs/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [259]:
x = df.drop("Time", axis=1)

In [260]:
linear_regression =  LinearRegression()

In [261]:
linear_regression.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [262]:
y_pred = linear_regression.predict(x)

In [263]:
y_pred

array([27.39611684, 60.53949051, 76.12014994, 47.93472917, 14.41533036,
       21.32633737, 10.09862436, 10.15627649, 43.40621408,  3.38992234,
       38.27964837, 53.23111462, 21.25389081, 41.67768696, 16.60520389,
       35.97610921, 72.38261404,  8.03291924, 40.77765429, 16.23059741,
       20.06362188, 54.41648927, 23.2612031 ,  9.88055557, 25.69099865,
       13.18305461, 23.94567809,  4.87956787, 53.82866278, 68.08901781,
       13.6705687 , 28.38330766, -8.63008798,  9.15270205,  9.80042074,
       37.85663886, -2.19962121, 13.92735924, 14.65450276, 12.2733935 ,
        9.98368652, 22.52310064, 34.0107387 , 14.42821816, 15.97541896,
       -6.27982723])